In [2]:
## Import modules 

import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

## read data 
CIP_data = pd.read_csv("CIP_Resistant_disagregated.csv")

print(CIP_data.columns)

Index(['Unnamed: 0', 'CLINIC', 'YEAR', 'GENDERSP', 'Susceptible'], dtype='object')


In [33]:
## Multivariable model with X1 = CLINIC and X2 = GENDERSP
X1 = CIP_data[["CLINIC"]]
X2 = CIP_data[["GENDERSP"]]
y = CIP_data[["Susceptible"]]


## OneHot encoding - CLINIC
encoder_loc = OneHotEncoder(drop='first') #drop='first' handles collinearity https://geoffruddock.com/one-hot-encoding-plus-linear-regression-equals-multi-collinearity/ BUT NOT WORKING 
encoder_categories_loc  = encoder_loc.fit(CIP_data[['CLINIC']]).categories_
#print(encoder_categories_loc)
encoder_categories_loc  = encoder_categories_loc[0].tolist()
#print(len(encoder_categories_loc))
encoded_loc = encoder_loc.fit_transform(CIP_data[['CLINIC']])
encoder_df_loc  = pd.DataFrame(encoder_loc.fit_transform(CIP_data[['CLINIC']]).toarray())
CIP_data_encoded_loc = CIP_data.join(encoder_df_loc)
#print(CIP_data_encoded_loc.shape)
CIP_data_encoded_loc.head(12)
col_names = list(CIP_data.columns) + encoder_categories_loc[1:] #for some reason when i get the categories it doesn't drop the first?
print(col_names)
CIP_data_encoded_loc.columns = col_names




['Unnamed: 0', 'CLINIC', 'YEAR', 'GENDERSP', 'Susceptible', 'ANC', 'ATL', 'BAL', 'BHM', 'BOS', 'BUF', 'CAM', 'CHI', 'CIN', 'CLE', 'COL', 'DAL', 'DEN', 'DTR', 'FBG', 'GRB', 'HON', 'IND', 'JAC', 'KCY', 'LAX', 'LBC', 'LVG', 'MIA', 'MIL', 'MIN', 'NOR', 'NYC', 'OKC', 'ORA', 'PHI', 'PHX', 'PON', 'POR', 'RIC', 'SDG', 'SEA', 'SFO', 'SLC', 'STL', 'WDC']


In [36]:
## OneHot encoding - GENDERSP
encoder_gen = OneHotEncoder(drop='first')
encoder_categories_gend  = encoder_gen.fit(CIP_data_encoded_loc[['GENDERSP']]).categories_
encoder_categories_gend  = encoder_categories_gend[0].tolist()
encoder_df_gend  = pd.DataFrame(encoder_gen.fit_transform(CIP_data_encoded_loc[['GENDERSP']]).toarray())
CIP_data_encoded_gend = CIP_data_encoded_loc.join(encoder_df_gend)
CIP_data_encoded_gend.head(12)
col_names = list(CIP_data_encoded_loc.columns) + encoder_categories_gend[1:]
#print(col_names)
CIP_data_encoded_gend.columns = col_names
#
print(len(list(CIP_data["GENDERSP"].unique())))
print(len(list(CIP_data["CLINIC"].unique())))


CIP_data_encoded_gend.head(2)




4
42


,Unnamed: 0,CLINIC,YEAR,GENDERSP,Susceptible,ANC,ATL,BAL,BHM,BOS,...,RIC,SDG,SEA,SFO,SLC,STL,WDC,MSMW,MSW,Oth/Unk/Missing
0,0,ALB,2000,MSW,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,ALB,2000,MSW,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [38]:
## save file

CIP_data_encoded = CIP_data_encoded_gend ## reassign for sake of straighforwardness 

CIP_data_encoded.to_csv("CIP_data_encoded.csv")

In [7]:
############### Need to redo encoding. No longer want each location to be an indiviudal 
############### factor; group into broader locations 
############### Also, add in (a) prevalence in region in previous year and (b) prevalence across clinic in previous year 
CIP_data = pd.read_csv("CIP_Resistant_disagregated.csv")

encoder_gen = OneHotEncoder(drop='first')
encoder_categories_gend  = encoder_gen.fit(CIP_data[['GENDERSP']]).categories_
encoder_categories_gend  = encoder_categories_gend[0].tolist()
encoder_df_gend  = pd.DataFrame(encoder_gen.fit_transform(CIP_data[['GENDERSP']]).toarray())
CIP_data_encoded_gend = CIP_data.join(encoder_df_gend)
CIP_data_encoded_gend.head(12)
col_names = list(CIP_data.columns) + encoder_categories_gend[1:]
#print(col_names)
CIP_data_encoded_gend.columns = col_names
#
print(CIP_data_encoded_gend.head())


######### Try "get dummies" method

#X = pd.get_dummies(CIP_data[['GENDERSP']])
#print(X.head())

   Unnamed: 0 CLINIC  YEAR GENDERSP  Susceptible  MSMW  MSW  Oth/Unk/Missing
0           0    ALB  2000      MSW            1   0.0  1.0              0.0
1           0    ALB  2000      MSW            1   0.0  1.0              0.0
2           0    ALB  2000      MSW            1   0.0  1.0              0.0
3           0    ALB  2000      MSW            1   0.0  1.0              0.0
4           0    ALB  2000      MSW            1   0.0  1.0              0.0
   GENDERSP_MSM  GENDERSP_MSMW  GENDERSP_MSW  GENDERSP_Oth/Unk/Missing
0             0              0             1                         0
1             0              0             1                         0
2             0              0             1                         0
3             0              0             1                         0
4             0              0             1                         0


In [14]:
###Now need to fix it so that things are coded by region 
### Coding

west = ['POR', 'PHX', 'HON', 'SDG', 'SFO', 'ANC', 'SEA', 'DEN', 'LVG', 'ORA', 'LBC', 'SLC', 'LAX']
southwest = ['OKC','MIN', 'ALB', 'DAL']
midwest = ['KCY','CHI', 'PON', 'CIN', 'JAC', 'IND', 'STL','DTR', 'MIL', 'COL', 'CLE']
southeast = ['GRB', 'NOR','WDC','MIA', 'BHM','FBG','ATL', 'RIC']
northeast = ['BUF','BOS', 'CAM', 'NYC', 'BAL', 'PHI']

### One-hot encoding 
CIP_data_encoded_gend['REGION'] = CIP_data_encoded_gend['CLINIC'].apply(lambda x: 
    'West' if (x in west) else (
        'Southwest' if (x in southwest) else(
            'Midwest' if (x in midwest) else(
                'Southeast' if (x in southeast) else(
                    'Northeast' if (x in northeast) else 'Other'))))) 

#print(CIP_data_encoded_gend.head())


encoder_region = OneHotEncoder(drop='first')
encoder_categories_region  = encoder_region.fit(CIP_data_encoded_gend[['REGION']]).categories_
encoder_categories_region  = encoder_categories_region[0].tolist()
encoder_df_region = pd.DataFrame(encoder_region.fit_transform(CIP_data_encoded_gend[['REGION']]).toarray())
CIP_data_encoded_gend_region = CIP_data_encoded_gend.join(encoder_df_region)
col_names = list(CIP_data_encoded_gend.columns) + encoder_categories_region[1:]
#print(col_names)
CIP_data_encoded_gend_region.columns = col_names
print(CIP_data_encoded_gend_region.head())


   Unnamed: 0 CLINIC  YEAR GENDERSP  Susceptible  MSMW  MSW  Oth/Unk/Missing  \
0           0    ALB  2000      MSW            1   0.0  1.0              0.0   
1           0    ALB  2000      MSW            1   0.0  1.0              0.0   
2           0    ALB  2000      MSW            1   0.0  1.0              0.0   
3           0    ALB  2000      MSW            1   0.0  1.0              0.0   
4           0    ALB  2000      MSW            1   0.0  1.0              0.0   

      Region     REGION  Northeast  Southeast  Southwest  West  
0  Southwest  Southwest        0.0        0.0        1.0   0.0  
1  Southwest  Southwest        0.0        0.0        1.0   0.0  
2  Southwest  Southwest        0.0        0.0        1.0   0.0  
3  Southwest  Southwest        0.0        0.0        1.0   0.0  
4  Southwest  Southwest        0.0        0.0        1.0   0.0  
